In [131]:
#import libs
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train = pd.read_csv('..input/zindi/data/zindi_data/trainOriginal.csv')
sub_sample=pd.read_csv('..input/zindi_data/SampleSubmission.csv')
def fix_data(input_set):
    input_set['Date'] = pd.to_datetime(input_set['Date'], infer_datetime_format=True) # change date
    input_set['Date'] = input_set.Date.dt.strftime("%m%d") # convert format to month-day 
    input_set['Date']  = input_set['Date'].astype(int) # convert to int
    return input_set
train = fix_data(train)

In [132]:
def fix_sub(df):
    df[['Territory','Date']] = df['Territory X Date'].str.split(' X ' , n=1 , expand=True)
    df=fix_data(df)
    return df   

In [133]:
test=fix_sub(sub_sample)



In [138]:
train['LConfirmedCases']=np.log1p(train['cases'])
train['LFatalities']=np.log1p(train['target'])
train['LDConfirmedCases']=train.groupby('Territory')[['LConfirmedCases']].diff()
train['LDFatalities']=train.groupby('Territory')[['LFatalities']].diff()

train['LConfirmedCases1']=train.groupby('Territory')[['LConfirmedCases']].shift(1)
train['LConfirmedCases2']=train.groupby('Territory')[['LConfirmedCases']].shift(2)
train['LConfirmedCases3']=train.groupby('Territory')[['LConfirmedCases']].shift(3)
train['LConfirmedCases4']=train.groupby('Territory')[['LConfirmedCases']].shift(4)
train['LDConfirmedCases1']=train.groupby('Territory')[['LDConfirmedCases']].shift(1)
train['LDConfirmedCases2']=train.groupby('Territory')[['LDConfirmedCases']].shift(2)
train['LDConfirmedCases3']=train.groupby('Territory')[['LDConfirmedCases']].shift(3)
train['LDConfirmedCases4']=train.groupby('Territory')[['LDConfirmedCases']].shift(4)
train['LDConfirmedCasesMA']=(train.LDConfirmedCases+train.LDConfirmedCases1+train.LDConfirmedCases2+train.LDConfirmedCases3+train.LDConfirmedCases4)/5
train['LDConfirmedCasesMA1']=train.groupby('Territory')[['LDConfirmedCasesMA']].shift(1)
train['LDConfirmedCasesMA2']=train.groupby('Territory')[['LDConfirmedCasesMA']].shift(2)
train['LDConfirmedCasesMA3']=train.groupby('Territory')[['LDConfirmedCasesMA']].shift(3)
train['LDConfirmedCasesMA4']=train.groupby('Territory')[['LDConfirmedCasesMA']].shift(4)
train['LConfirmedCasesMA']=(train.LConfirmedCases+train.LConfirmedCases1+train.LConfirmedCases2+train.LConfirmedCases3+train.LConfirmedCases4)/5
train['LConfirmedCasesMA1']=train.groupby('Territory')[['LConfirmedCasesMA']].shift(1)
train['LConfirmedCasesMA2']=train.groupby('Territory')[['LConfirmedCasesMA']].shift(2)
train['LConfirmedCasesMA3']=train.groupby('Territory')[['LConfirmedCasesMA']].shift(3)
train['LConfirmedCasesMA4']=train.groupby('Territory')[['LConfirmedCasesMA']].shift(4)

train['LFatalities1']=train.groupby('Territory')[['LFatalities']].shift(1)
train['LFatalities2']=train.groupby('Territory')[['LFatalities']].shift(2)
train['LFatalities3']=train.groupby('Territory')[['LFatalities']].shift(3)
train['LFatalities4']=train.groupby('Territory')[['LFatalities']].shift(4)
train['LDFatalities1']=train.groupby('Territory')[['LDFatalities']].shift(1)
train['LDFatalities2']=train.groupby('Territory')[['LDFatalities']].shift(2)
train['LDFatalities3']=train.groupby('Territory')[['LDFatalities']].shift(3)
train['LDFatalities4']=train.groupby('Territory')[['LDFatalities']].shift(4)
train['LDFatalitiesMA']=(train.LDFatalities+train.LDFatalities1+train.LDFatalities2+train.LDFatalities3+train.LDFatalities4)/5
train['LDFatalitiesMA1']=train.groupby('Territory')[['LDFatalitiesMA']].shift(1)
train['LDFatalitiesMA2']=train.groupby('Territory')[['LDFatalitiesMA']].shift(2)
train['LDFatalitiesMA3']=train.groupby('Territory')[['LDFatalitiesMA']].shift(3)
train['LDFatalitiesMA4']=train.groupby('Territory')[['LDFatalitiesMA']].shift(4)
train['LFatalitiesMA']=(train.LFatalities+train.LFatalities1+train.LFatalities2+train.LFatalities3+train.LFatalities4)/5
train['LFatalitiesMA1']=train.groupby('Territory')[['LFatalitiesMA']].shift(1)
train['LFatalitiesMA2']=train.groupby('Territory')[['LFatalitiesMA']].shift(2)
train['LFatalitiesMA3']=train.groupby('Territory')[['LFatalitiesMA']].shift(3)
train['LFatalitiesMA4']=train.groupby('Territory')[['LFatalitiesMA']].shift(4)
train['serd']=train.groupby('Territory').cumcount()
train.loc[train.cases==0,'days_since_confirmed']=0
train.loc[train.cases>0,'days_since_confirmed']=train[train.cases>0].groupby('Territory').cumcount() #The first is 0 to avoid leakakge

In [140]:
from lightgbm import LGBMRegressor

lgbm_cc=LGBMRegressor(num_leaves = 85,learning_rate =10**-1.89,n_estimators=1000,min_sum_hessian_in_leaf=(10**-4.1),min_child_samples =2,subsample =0.97,subsample_freq=10,
                   colsample_bytree = 0.68,reg_lambda=10**1.4,random_state=1234,n_jobs=4)
lgbm_f=LGBMRegressor(num_leaves = 1000,learning_rate =10**-1.63,n_estimators=1000,min_sum_hessian_in_leaf=(10**-4.04),min_child_samples =14,subsample =0.66,subsample_freq=5,
                   colsample_bytree = 0.8,reg_lambda=10**1.92,random_state=1234,n_jobs=4)

from sklearn.preprocessing import LabelEncoder
oe=LabelEncoder()
X=oe.fit_transform(train['Territory'])
train['CR']=X
#train=train.drop('Territory',axis=1)
#train['PS']=X[:,1]
train=train.fillna(0)
lgbm_cc.fit(train.loc[:,['LDConfirmedCases1','LDConfirmedCases2','LDConfirmedCases3','LDConfirmedCases4',
                                  'LDFatalities1','LDFatalities2','LDFatalities3','LDFatalities4','days_since_confirmed','CR',
                                 'LDConfirmedCasesMA1','LDConfirmedCasesMA2','LDConfirmedCasesMA3','LDConfirmedCasesMA4',
                                 'LDFatalitiesMA1','LDFatalitiesMA2','LDFatalitiesMA3','LDFatalitiesMA4',
                                     'LConfirmedCases1','LConfirmedCases2','LConfirmedCases3','LConfirmedCases4',
                                  'LFatalities1','LFatalities2','LFatalities3','LFatalities4',
                                 'LConfirmedCasesMA1','LConfirmedCasesMA2','LConfirmedCasesMA3','LConfirmedCasesMA4',
                                 'LFatalitiesMA1','LFatalitiesMA2','LFatalitiesMA3','LFatalitiesMA4']],train.LDConfirmedCases,categorical_feature=['CR'])
lgbm_f.fit(train.loc[:,['LDConfirmedCases1','LDConfirmedCases2','LDConfirmedCases3','LDConfirmedCases4',
                                  'LDFatalities1','LDFatalities2','LDFatalities3','LDFatalities4','days_since_confirmed','CR',
                                 'LDConfirmedCasesMA1','LDConfirmedCasesMA2','LDConfirmedCasesMA3','LDConfirmedCasesMA4',
                                 'LDFatalitiesMA1','LDFatalitiesMA2','LDFatalitiesMA3','LDFatalitiesMA4',
                                     'LConfirmedCases1','LConfirmedCases2','LConfirmedCases3','LConfirmedCases4',
                                  'LFatalities1','LFatalities2','LFatalities3','LFatalities4',
                                 'LConfirmedCasesMA1','LConfirmedCasesMA2','LConfirmedCasesMA3','LConfirmedCasesMA4',
                                 'LFatalitiesMA1','LFatalitiesMA2','LFatalitiesMA3','LFatalitiesMA4','LConfirmedCases','LDConfirmedCases']],train.LDFatalities,categorical_feature=['CR'])   

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
              importance_type='split', learning_rate=0.023442288153199226,
              max_depth=-1, min_child_samples=14, min_child_weight=0.001,
              min_split_gain=0.0, min_sum_hessian_in_leaf=9.120108393559096e-05,
              n_estimators=1000, n_jobs=4, num_leaves=1000, objective=None,
              random_state=1234, reg_alpha=0.0, reg_lambda=83.17637711026708,
              silent=True, subsample=0.66, subsample_for_bin=200000,
              subsample_freq=5)

In [141]:
from sklearn.metrics import mean_squared_log_error
train['serd']=train.groupby('Territory').cumcount()
trainpred = pd.concat((train,test[test.Date>train.Date.max()])).reset_index(drop=True)
trainpred.sort_values(['Territory','Date'],inplace=True)
X=oe.transform(trainpred['Territory'])
trainpred['CR']=X
trainpred['serd']=trainpred.groupby('Territory').cumcount()
trainpred.loc[trainpred.cases.isnull(),'cases']=1 #Heuristic
trainpred.loc[trainpred.cases==0,'days_since_confirmed']=0
trainpred.loc[trainpred.cases>0,'days_since_confirmed']=trainpred[trainpred.cases>0].groupby('Territory').cumcount() #The first is 0 to avoid leakakge
trainpred['LConfirmedCases']=np.log1p(trainpred['cases'])
trainpred['LFatalities']=np.log1p(trainpred['target'])
trainpred['LDConfirmedCases']=trainpred.groupby('Territory')[['LConfirmedCases']].diff()
trainpred['LDFatalities']=trainpred.groupby('Territory')[['LFatalities']].diff()

for serd in range(train.serd.max()+1,trainpred.serd.max()+1):
    print(serd)
    trainpred['LConfirmedCases1']=trainpred.groupby('Territory')[['LConfirmedCases']].shift(1)
    trainpred['LConfirmedCases2']=trainpred.groupby('Territory')[['LConfirmedCases']].shift(2)
    trainpred['LConfirmedCases3']=trainpred.groupby('Territory')[['LConfirmedCases']].shift(3)
    trainpred['LConfirmedCases4']=trainpred.groupby('Territory')[['LConfirmedCases']].shift(4)
    trainpred['LDConfirmedCases1']=trainpred.groupby('Territory')[['LDConfirmedCases']].shift(1)
    trainpred['LDConfirmedCases2']=trainpred.groupby('Territory')[['LDConfirmedCases']].shift(2)
    trainpred['LDConfirmedCases3']=trainpred.groupby('Territory')[['LDConfirmedCases']].shift(3)
    trainpred['LDConfirmedCases4']=trainpred.groupby('Territory')[['LDConfirmedCases']].shift(4)
    trainpred['LDConfirmedCasesMA']=(trainpred.LDConfirmedCases+trainpred.LDConfirmedCases1+trainpred.LDConfirmedCases2+trainpred.LDConfirmedCases3+trainpred.LDConfirmedCases4)/5
    trainpred['LDConfirmedCasesMA1']=trainpred.groupby('Territory')[['LDConfirmedCasesMA']].shift(1)
    trainpred['LDConfirmedCasesMA2']=trainpred.groupby('Territory')[['LDConfirmedCasesMA']].shift(2)
    trainpred['LDConfirmedCasesMA3']=trainpred.groupby('Territory')[['LDConfirmedCasesMA']].shift(3)
    trainpred['LDConfirmedCasesMA4']=trainpred.groupby('Territory')[['LDConfirmedCasesMA']].shift(4)
    trainpred['LConfirmedCasesMA']=(trainpred.LConfirmedCases+trainpred.LConfirmedCases1+trainpred.LConfirmedCases2+trainpred.LConfirmedCases3+trainpred.LConfirmedCases4)/5
    trainpred['LConfirmedCasesMA1']=trainpred.groupby('Territory')[['LConfirmedCasesMA']].shift(1)
    trainpred['LConfirmedCasesMA2']=trainpred.groupby('Territory')[['LConfirmedCasesMA']].shift(2)
    trainpred['LConfirmedCasesMA3']=trainpred.groupby('Territory')[['LConfirmedCasesMA']].shift(3)
    trainpred['LConfirmedCasesMA4']=trainpred.groupby('Territory')[['LConfirmedCasesMA']].shift(4)

    trainpred['LFatalities1']=trainpred.groupby('Territory')[['LFatalities']].shift(1)
    trainpred['LFatalities2']=trainpred.groupby('Territory')[['LFatalities']].shift(2)
    trainpred['LFatalities3']=trainpred.groupby('Territory')[['LFatalities']].shift(3)
    trainpred['LFatalities4']=trainpred.groupby('Territory')[['LFatalities']].shift(4)
    trainpred['LDFatalities1']=trainpred.groupby('Territory')[['LDFatalities']].shift(1)
    trainpred['LDFatalities2']=trainpred.groupby('Territory')[['LDFatalities']].shift(2)
    trainpred['LDFatalities3']=trainpred.groupby('Territory')[['LDFatalities']].shift(3)
    trainpred['LDFatalities4']=trainpred.groupby('Territory')[['LDFatalities']].shift(4)
    trainpred['LDFatalitiesMA']=(trainpred.LDFatalities+trainpred.LDFatalities1+trainpred.LDFatalities2+trainpred.LDFatalities3+trainpred.LDFatalities4)/5
    trainpred['LDFatalitiesMA1']=trainpred.groupby('Territory')[['LDFatalitiesMA']].shift(1)
    trainpred['LDFatalitiesMA2']=trainpred.groupby('Territory')[['LDFatalitiesMA']].shift(2)
    trainpred['LDFatalitiesMA3']=trainpred.groupby('Territory')[['LDFatalitiesMA']].shift(3)
    trainpred['LDFatalitiesMA4']=trainpred.groupby('Territory')[['LDFatalitiesMA']].shift(4)
    trainpred['LFatalitiesMA']=(trainpred.LFatalities+trainpred.LFatalities1+trainpred.LFatalities2+trainpred.LFatalities3+trainpred.LFatalities4)/5
    trainpred['LFatalitiesMA1']=trainpred.groupby('Territory')[['LFatalitiesMA']].shift(1)
    trainpred['LFatalitiesMA2']=trainpred.groupby('Territory')[['LFatalitiesMA']].shift(2)
    trainpred['LFatalitiesMA3']=trainpred.groupby('Territory')[['LFatalitiesMA']].shift(3)
    trainpred['LFatalitiesMA4']=trainpred.groupby('Territory')[['LFatalitiesMA']].shift(4)
    trainpred.loc[trainpred.serd==serd,'LDConfirmedCases']= lgbm_cc.predict(trainpred.loc[trainpred.serd==serd,['LDConfirmedCases1','LDConfirmedCases2','LDConfirmedCases3','LDConfirmedCases4',
                                  'LDFatalities1','LDFatalities2','LDFatalities3','LDFatalities4','days_since_confirmed','CR',
                                 'LDConfirmedCasesMA1','LDConfirmedCasesMA2','LDConfirmedCasesMA3','LDConfirmedCasesMA4',
                                 'LDFatalitiesMA1','LDFatalitiesMA2','LDFatalitiesMA3','LDFatalitiesMA4',
                                     'LConfirmedCases1','LConfirmedCases2','LConfirmedCases3','LConfirmedCases4',
                                  'LFatalities1','LFatalities2','LFatalities3','LFatalities4',
                                 'LConfirmedCasesMA1','LConfirmedCasesMA2','LConfirmedCasesMA3','LConfirmedCasesMA4',
                                 'LFatalitiesMA1','LFatalitiesMA2','LFatalitiesMA3','LFatalitiesMA4']])
    trainpred.loc[(trainpred.serd==serd) & (trainpred.LDConfirmedCases<0),'LDConfirmedCases']=0
    trainpred.loc[trainpred.serd==serd,'LConfirmedCases']=trainpred.loc[trainpred.serd==serd,'LDConfirmedCases']+trainpred.loc[trainpred.serd==serd,'LConfirmedCases1']
    trainpred.loc[trainpred.serd==serd,'cases']=np.exp(trainpred.loc[trainpred.serd==serd,'LConfirmedCases'])-1
    
    trainpred.loc[trainpred.serd==serd,'LDFatalities']= lgbm_f.predict(trainpred.loc[trainpred.serd==serd,['LDConfirmedCases1','LDConfirmedCases2','LDConfirmedCases3','LDConfirmedCases4',
                                  'LDFatalities1','LDFatalities2','LDFatalities3','LDFatalities4','days_since_confirmed','CR',
                                 'LDConfirmedCasesMA1','LDConfirmedCasesMA2','LDConfirmedCasesMA3','LDConfirmedCasesMA4',
                                 'LDFatalitiesMA1','LDFatalitiesMA2','LDFatalitiesMA3','LDFatalitiesMA4',
                                     'LConfirmedCases1','LConfirmedCases2','LConfirmedCases3','LConfirmedCases4',
                                  'LFatalities1','LFatalities2','LFatalities3','LFatalities4',
                                 'LConfirmedCasesMA1','LConfirmedCasesMA2','LConfirmedCasesMA3','LConfirmedCasesMA4',
                                 'LFatalitiesMA1','LFatalitiesMA2','LFatalitiesMA3','LFatalitiesMA4','LConfirmedCases','LDConfirmedCases']])
    trainpred.loc[(trainpred.serd==serd) & (trainpred.LDFatalities<0),'LDFatalities']=0
    trainpred.loc[trainpred.serd==serd,'LFatalities']=trainpred.loc[trainpred.serd==serd,'LDFatalities']+trainpred.loc[trainpred.serd==serd,'LFatalities1']
    trainpred.loc[trainpred.serd==serd,'target']=np.exp(trainpred.loc[trainpred.serd==serd,'LFatalities'])-1

86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137


In [130]:
trainpred.loc[(trainpred.Date<=max(test.Date)) & (trainpred.Date>=min(test.Date)),'Territory X Date']=test.loc[:,'Territory X Date'].values
submission=trainpred.loc[(trainpred.Date>=min(test.Date))  ][['Territory X Date','cases','target']]
sub=submission.drop('cases',axis=1)
sub.to_csv("submission5.csv",index=False)


MAE:  30.371082362108595
